<a href="https://colab.research.google.com/github/ssundar6087/Deep-Learning-Mini-Course/blob/main/Pytorch/DL_Minicourse_Pytorch_Day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Deconstructing the Model Part 1
Today, we'll look at the model. Note that 90% of the code will be the same as the previous notebook. Our focus is to play with the model architecture and see if we can improve results.

# Image Classification Pytorch

In [ ]:
# Imports
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

## **YOUR EXERCISE HERE: Define the Model  👇** 
In today's exercise, you'll be creating a convolutional neural network. Previously, we flatten a 3 channel image (32 x 32 x 3) into a vector and passed that through linear layers. It's now time to spruce up the model and pass the image directly through convolutional layers.

**Convolutions?**
The example below is for 2D convolutions, but generally, you have as many dimensions as you have in the input stream (images, audio, time-series, point clouds, etc.). $B$ is the batch size and $C$ is the number of channels. $dim$ is the resolution of the input which in this case is 32. Finally, $k$ is the size of the convolutional kernel.

  - What is my input shape: $B \times C \times dim \times dim$ 

  - What is my output shape: $B \times C \times dim' \times dim'$

  - Here $dim' = \frac{(dim + 2 \times padding - k)}{stride} + 1$

**Potential Pitfall**

Watch out for the transition from the convolutional layers to the linear layers. You will have to compute the dimensions of the _flattened_ tensor at the end of convolutions.


### HINT: DO NOT USE UNTIL YOU'VE TRIED IT YOURSELF
👉[Convolutional Networks In Pytorch](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) 👈

In [ ]:
class BabyThanos(nn.Module):
  def __init__(self, in_dims, in_channels, n_classes=10):
    super().__init__()
    
    self.in_dims = in_dims 
    self.in_channels = in_channels
    self.n_classes = n_classes
    # define the layers here
  
  
  def forward(self, x):
    # define the flow of data through the layers here
    
    return x
    

In [ ]:
IN_DIMS = 32 
IN_CHANNELS = 3
N_CLASSES = 10
net = BabyThanos(in_dims=IN_DIMS, in_channels=IN_CHANNELS, n_classes=N_CHANNELS)

In [ ]:
print(net)

## Hyperparameters 
**Note:** Use the best values from the previous exercise

In [ ]:
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 1e-3

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Define Optimizer & Loss Function
These two functions allow us to help baby thanos learn from his mistakes.

In [ ]:
criterion = nn.CrossEntropyLoss() # Loss Function
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE) # Optimizer 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Train and Evaluate the Network
The training and validation loops call the same set of functions over and over, so we'll package them into separate functions. Note that the validation loop does not have any optimizer calls. 

In [ ]:
from tqdm.notebook import tqdm
def train_step(model, train_loader, optimizer, criterion):
  model.train()
  epoch_loss = []
  total, correct = 0, 0

  for i, batch in tqdm(enumerate(train_loader), 
                       total=len(train_loader),
                       leave=False,
                       ):
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad() # Erase history - clean slate

    predictions = model(images) # forward -> model (images) -> make guesses on labels
    loss = criterion(predictions, labels) # how did I do?
    epoch_loss.append(loss.item())
    _, predicted = torch.max(predictions.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item() # Accuracy score
    loss.backward() # backward pass
    optimizer.step() # Update the weights using gradients
  
  return np.mean(epoch_loss), correct / total


In [ ]:
def valid_step(model, val_loader, criterion):
  model.eval()
  epoch_loss = []
  total, correct = 0, 0

  with torch.no_grad():
    for i, batch in tqdm(enumerate(val_loader), 
                        total=len(val_loader),
                        leave=False,
                        ):
      images, labels = batch
      images = images.to(device)
      labels = labels.to(device)
      # Note that there's no optimizer here
      predictions = model(images)
      loss = criterion(predictions, labels)
      epoch_loss.append(loss.item())
      _, predicted = torch.max(predictions.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  
  return np.mean(epoch_loss), correct / total

In [ ]:
net = net.to(device)
losses = {"train_loss": [], "val_loss": []}
accuracies = {"train_acc": [], "val_acc": []}
epochs = []
for epoch in tqdm(range(EPOCHS), total=EPOCHS):
  train_loss, train_acc = train_step(net, 
                                     trainloader, 
                                     optimizer, 
                                     criterion,)
  
  val_loss, val_acc = valid_step(net, 
                                 testloader, 
                                 criterion,
                                 )
  
  losses["train_loss"].append(train_loss)
  losses["val_loss"].append(val_loss)
  accuracies["train_acc"].append(train_acc)
  accuracies["val_acc"].append(val_acc)
  epochs.append(epoch)

  print(f'[{epoch + 1}] train loss: {train_loss}  train accuracy: {train_acc}  val loss: {val_loss}  val accuracy: {val_acc}')

## Plot the Loss and Accuracy of our Model

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(epochs, losses["train_loss"], label="train")
plt.plot(epochs, losses["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid("on")
plt.legend()
plt.title("Loss vs Epochs")

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(epochs, accuracies["train_acc"], label="train")
plt.plot(epochs, accuracies["val_acc"], label="val")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid("on")
plt.legend()
plt.title("Accuracy vs Epochs")